## Competition Overview
*Description*

- The downfield pass is the crown jewel of American sports. When the ball is in the air, anything can happen, like a touchdown, an interception, or a contested catch. The uncertainty and the importance of the outcome of these plays is what helps keep audiences on the edge of its seat.

- The 2026 Big Data Bowl is designed to help the National Football League better understand player movement during the pass play, starting with when the ball is thrown and ending when the ball is either caught or ruled incomplete. For the offensive team, this means focusing on the targeted receiver, whose job is to move towards the ball landing location in order to complete a catch. For the defensive team, who could have several players moving towards the ball, their jobs are to both prevent the offensive player from making a catch, while also going for the ball themselves. This year's Big Data Bowl asks our fans to help track the movement of these players.

- In the Prediction Competition of the Big Data Bowl, participants are tasked with predicting player movement with the ball in the air. Specifically, the NFL is sharing data before the ball is thrown (including the Next Gen Stats tracking data), and stopping the play the moment the quarterback releases the ball. In addition to the pre-pass tracking data, we are providing participants with which offensive player was targeted (e.g, the targeted receiver) and the landing location of the pass.

- Using the information above, participants should generate prediction models for player movement during the frames when the ball is in the air. The most accurate algorithms will be those whose output most closely matches the eventual player movement of each player.

*Competition Specifics*
- In the NFL's tracking data, there are 10 frames per second. As a result, if a ball is in the air for 2.5 seconds, there will be 25 frames of location data to predict.

- Quick passes (less than half a second), deflected passes, and throwaway passes are dropped from the competition.

- Evaluation for the training data is based on historical data. Evaluation for the leaderboard is based on data that hasn't happened yet. Specifically, we will be doing a live leaderboard covering the last five weeks of the 2025 NFL season.

*Evaluation*
- Submissions are evaluated using the Root Mean Squared Error between the predicted and the observed target.

### Initial Data Load for Competition

In [ ]:
import pandas as pd
import numpy as np
import glob

# Training Data Materialization
# Change to match your directory's pathh
train_path = r"C:\Users\F4WF76A\OneDrive - Fiserv Corp\Repos\NFL-Big-Data-Bowl-Prediction\Data\train"
input_files = sorted(glob.glob(train_path + "/input_2023_w*.csv"))
output_files = sorted(glob.glob(train_path + "/output_2023_w*.csv"))
# Model training features dataframe
train_feature_df = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
# Training target dataframe
train_target_df = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)


# Test Data Materialization
# Test input dataframe with features used to train model
test_input_df = pd.read_csv('Data/test_input.csv')
# Test ground truth data to compare the predictions against
test_df = pd.read_csv('Data/test.csv')

### Training Data

**Training Input/Features**
- From files with pattern 'input_2023_w[01-18].csv'
- The input data contains tracking data before the pass is thrown

*Data Dictionary*
- game_id: Game identifier, unique (numeric)
- play_id: Play identifier, not unique across games (numeric)
- player_to_predict: whether or not the x/y prediction for this player will be scored (bool)
- nfl_id: Player identification number, unique across players (numeric)
- frame_id: Frame identifier for each play/type, starting at 1 for each game_id/play_id/file type (input or output) (numeric)
- play_direction: Direction that the offense is moving (left or right)
- absolute_yardline_number: Distance from end zone for possession team (numeric)
- player_name: player name (text)
- player_height: player height (ft-in)
- player_weight: player weight (lbs)
- player_birth_date: birth date (yyyy-mm-dd)
- player_position: the player's position (the specific role on the field that they typically play)
- player_side: team player is on (Offense or Defense)
- player_role: role player has on play (Defensive Coverage, Targeted Receiver, Passer or Other Route Runner)
- x: Player position along the long axis of the field, generally within 0 - 120 yards. (numeric)
- y: Player position along the short axis of the field, generally within 0 - 53.3 yards. (numeric)
- s: Speed in yards/second (numeric)
- a: Acceleration in yards/second^2 (numeric)
- o: orientation of player (deg)
- dir: angle of player motion (deg)
- num_frames_output: Number of frames to predict in output data for the given game_id/play_id/nfl_id. (numeric)
- ball_land_x: Ball landing position position along the long axis of the field, generally within 0 - 120 yards. (numeric)
- ball_land_y: Ball landing position along the short axis of the field, generally within 0 - 53.3 yards. (numeric)

In [ ]:
print('Train Features Dataframe Dimensions:', train_feature_df.shape)
# pd.set_option('display.max_columns', None)
train_feature_df.head()

**Training Target Data (Has Values We Want to Predict)**
- From files with pattern 'output_2023_w[01-18].csv'
- The output data contains tracking data after the pass is thrown.

*Data Dictionary*
- game_id: Game identifier, unique (numeric)
- play_id: Play identifier, not unique across games (numeric)
- nfl_id: Player identification number, unique across players. (numeric)
- frame_id: Frame identifier for each play/type, starting at 1 for each game_id/play_id/ file type (input or output). The maximum value for a given game_id, play_id and nfl_id will be the same as the num_frames_output value from the corresponding input file. (numeric)
- x: Player position along the long axis of the field, generally within 0-120 yards. (TARGET TO PREDICT)
- y: Player position along the short axis of the field, generally within 0 - 53.3 yards. (TARGET TO PREDICT)

In [ ]:
print('Train Target Dataframe Dimensions: ', train_target_df.shape)
train_target_df.head()

### Test Data

**Test Input/Features**
- Player tracking data at the same play as prediction. This file is provided only for convenience, the actual test data will be provided by the API.

In [ ]:
print('Test input Dataframe Dimensions:', test_input_df.shape)
test_input_df.head()

**Test Target**
- A mock test set representing the structure of the unseen test set. This file is provided only for convenience, the actual test_input data will be provided by the API. Contains the prediction targets as rows with columns (game_id, play_id, nfl_id, frame_id) representing each position that needs to be predicted.

In [ ]:
print('Test Target (Ground Truth) Dataframe Dimensions:', test_df.shape)
test_df.head()